In [1]:
import requests
import time
from bs4 import BeautifulSoup
from io import StringIO

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
html_content = requests.get(standings_url).text

In [4]:
soup = BeautifulSoup(html_content)
standings_table = soup.select('table.stats_table')[0]

In [5]:
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
html_content = requests.get(team_urls[0]).text

In [8]:
import pandas as pd
matches = pd.read_html(StringIO(html_content), match="Scores & Fixtures")[0]

In [9]:
soup = BeautifulSoup(html_content)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
html_content = requests.get(f"https://fbref.com{links[0]}").text

In [11]:
shooting = pd.read_html(StringIO(html_content), match="Shooting")[0]

In [12]:
shooting.head()

For Manchester City                                                        \
                 Date   Time                 Comp                Round  Day   
0          2024-08-10  15:00  FA Community Shield  FA Community Shield  Sat   
1          2024-08-18  16:30       Premier League          Matchweek 1  Sun   
2          2024-08-24  15:00       Premier League          Matchweek 2  Sat   
3          2024-08-31  17:30       Premier League          Matchweek 3  Sat   
4          2024-09-14  15:00       Premier League          Matchweek 4  Sat   

                                              ... Standard                \
  Venue Result     GF     GA        Opponent  ...     Dist   FK PK PKatt   
0  Home      D  1 (7)  1 (6)  Manchester Utd  ...      NaN  NaN  0     0   
1  Away      W      2      0         Chelsea  ...     19.1  0.0  0     0   
2  Home      W      4      1    Ipswich Town  ...     17.8  1.0  1     1   
3  Away      W      3      1        West Ham  ...     15.0  1.0  0     0   
4  Home      W      2      1       Brentford  ...     17.2  0.0  0     0   

  Expected                           Unnamed: 25_level_0  
        xG npxG npxG/Sh G-xG np:G-xG        Match Report  
0      NaN  NaN     NaN  NaN     NaN        Match Report  
1      0.8  0.8    0.07  1.2     1.2        Match Report  
2      3.3  2.6    0.20  0.7     0.4        Match Report  
3      3.0  3.0    0.13  0.0     0.0        Match Report  
4      2.1  2.1    0.12 -0.1    -0.1        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-10,15:00,FA Community Shield,FA Community Shield,Sat,Home,D,1 (7),1 (6),Manchester Utd,...,4-2-3-1,Jarred Gillett,Match Report,Manchester City won on penalty kicks following...,9,1,NaN,NaN,0,0
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,4-2-3-1,Anthony Taylor,Match Report,NaN,11,5,19.1,0.0,0,0
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,5-4-1,Samuel Allison,Match Report,NaN,13,4,17.8,1.0,1,1
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,4-2-3-1,Michael Oliver,Match Report,NaN,23,8,15.0,1.0,0,0
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2,1,Brentford,...,5-3-2,Darren Bond,Match Report,NaN,18,7,17.2,0.0,0,0


In [16]:
years = list(range(2024, 2022, -1))
all_matches = []

In [17]:
for year in years:
    content = requests.get(standings_url).text
    soup = BeautifulSoup(content)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        content = requests.get(team_url).text
        
        matches = pd.read_html(StringIO(content), match="Scores & Fixtures")[0]
        
        soup = BeautifulSoup(content)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        content = requests.get(f"https://fbref.com{links[0]}").text
        
        shooting = pd.read_html(StringIO(content), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(20)

In [18]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

In [19]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,Match Report,NaN,11.0,5.0,19.1,0.0,0,0,2024,Manchester City
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,Match Report,NaN,13.0,4.0,17.8,1.0,1,1,2024,Manchester City
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,23.0,8.0,15.0,1.0,0,0,2024,Manchester City
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2,1,Brentford,...,Match Report,NaN,18.0,7.0,17.2,0.0,0,0,2024,Manchester City
6,2024-09-22,16:30,Premier League,Matchweek 5,Sun,Home,D,2,2,Arsenal,...,Match Report,NaN,33.0,11.0,20.2,1.0,0,0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,Match Report,NaN,10.0,4.0,17.8,1.0,0,0,2023,Sheffield United
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,Match Report,NaN,15.0,4.0,13.5,0.0,0,0,2023,Sheffield United
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,Match Report,NaN,16.0,4.0,18.0,0.0,1,1,2023,Sheffield United
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,Match Report,NaN,13.0,1.0,21.0,0.0,0,0,2023,Sheffield United


In [20]:
match_df.to_csv("matches.csv")